<a href="https://colab.research.google.com/github/Pyfin5/Machine_Learning/blob/main/Reddit_NER_single_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Colab-Notebooks/Reddit_listening/

/content/drive/MyDrive/Colab-Notebooks/Reddit_listening


In [4]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [5]:
!pip install \
    praw \
    pandas \
    numpy \
    python-dotenv \
    datetime \
    IPython \
    llama_index \
    langchain_google_vertexai \
    google-cloud-aiplatform \
    praw \
    textwrap3 \
    langextract \
    asyncpraw

INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 88.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 73.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.2 MB/s  0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully

In [6]:
import praw
import pandas as pd
import os
import numpy as np
from dotenv import load_dotenv
import datetime as dt
import pandas as pd  # Data manipulation library
import sys
from IPython.display import Markdown, display
import pandas as pd
from dotenv import load_dotenv
from langchain_google_vertexai import VertexAIEmbeddings
from llama_index.core.llms import ChatMessage
from google.cloud import aiplatform
from praw.models import MoreComments
import textwrap
import langextract as lx
from google.colab import userdata
import asyncpraw as asyncpraw

In [7]:
load_dotenv() # Load environment variables from .env file

reddit = praw.Reddit(client_id=userdata.get("client_id"),
                     client_secret=userdata.get("client_secret"),
                     user_agent=userdata.get("user_agent"),
                     username=userdata.get("username"),
                     password=userdata.get("password"))


# Subreddit to scrape
subreddit = reddit.subreddit('Learnprogramming')

# Display the name of the Subreddit
print("Display Name:", subreddit.display_name)

Display Name: Learnprogramming


In [8]:
submission = reddit.submission(url = "https://www.reddit.com/r/learnprogramming/comments/vcpqq6/whats_up_with_linux_and_software_developers_if_i/")


In [9]:
print(submission)

vcpqq6


In [14]:
# make sure to extract all operating systems mentioned in the post. Each post might contain mention of multiple operating systems, extract them all.
# if you miss an extraction, you will be spanked.

system_prompt = "Extract operating system mentioned by the post author. Make sure to provide all instances of operating systems mentioned in the post."  \
"Each post might contain mention of multiple operating systems, extract them all including multple instances of the same operating system."

examples = [
            lx.data.ExampleData(
                text=   "I've used Linux for 18 years. For me, Linux is cleaner, easier to install, maintain and update, and is just faster — and there are no ads popping up." \
                        "That said, I don't play video games and I don't know the current state of video games on Linux. In the past, when people asked me if Linux was for them, I would ask two questions..." \
                        "As for programming (which I don't do) I think Linux would be a great platform for application development."
                        ,
                extractions=[
                    lx.data.Extraction(extraction_class="distribution", extraction_text="Linux", attributes = {'User':'Yes'})
                ]
            ),
            lx.data.ExampleData(
                text=   "I've always been a Windows user. I bought and used a windows for the first time professionally a few months ago " \
                "Learn to program on whatever you're familiar with. It's just easier.",
                extractions=[
                    lx.data.Extraction(extraction_class="distribution", extraction_text="Windows", attributes = {'User':'Yes'})
                ],
            ),
            lx.data.ExampleData( #New example which is not a user.
                text=   "Also, don't forget that running a Windows server is expensive. Linux is free." \
                "And you probably want to develop on the same or similar OS to what you'll be running on production. ",
                extractions=[
                    lx.data.Extraction(extraction_class="distribution", extraction_text="Windows", attributes = {'User':'No'})
                ]
            )
        ]



In [21]:
def extract_entity(input_text, NER_examples, system_prompt, text_name):

        # Define example data with entities in order of appearance
        examples = NER_examples

        result = lx.extract(
            text_or_documents=input_text,
            prompt_description=system_prompt,
            examples=examples,
            model_id="gemini-2.5-flash",
            api_key=userdata.get('GOOGLE_AI_STUDIO_API_KEY')
        )

        # Display entities with positions
        print(f"Input: {input_text}\n")
        print("Extracted entities:")
        for entity in result.extractions:
            position_info = ""
            if entity.char_interval:
                start, end = entity.char_interval.start_pos, entity.char_interval.end_pos
                position_info = f" (pos: {start}-{end})"
            print(f"• {entity.extraction_class.capitalize()}: {entity.extraction_text}{position_info}")
            print(f"  Attributes: {entity.attributes}") #Add print statement


        print(result)
        # Save and visualize the results
        lx.io.save_annotated_documents([result], output_name="{}.jsonl".format(text_name), output_dir=".")

        # Generate the interactive visualization
        html_content = lx.visualize("{}.jsonl".format(text_name))
        with open("{}.html".format(text_name), "w") as f:
          if hasattr(html_content, 'data'):
            f.write(html_content.data)
          else:
            f.write(html_content)

        print("Interactive visualization saved to {}visualization.html".format(text_name))




In [22]:
input_text = "Getting the same up and running (and stable!) in windows is a lot of work, especially if you want it to be smooth.. Linux is more configurable, and it is what most of the world's infrastructure and systems run on." \
"And yes, Linux is generally much more efficient than Windows." \
"That said, these days it is mainly preference." \
"For example, I prefer Linux so that I can configure it exactly how I want." \
"I like to use I3wm (A tiling wm) with a custom set of commands combined with oh-my-fish with my custom dotfiles."\
"I have also used MacOS but only minimally, liked it when I used it for a bit."

In [23]:
dist_extract = extract_entity(system_prompt=system_prompt, NER_examples = examples, input_text = input_text, text_name = 'linux_extract')

2025-08-18 23:43:36,853 - langextract.debug - DEBUG - [langextract.inference] CALL: BaseLanguageModel.__init__(self=<GeminiLanguageModel>, constraint=Constraint(co...NONE: 'none'>), kwargs={})
2025-08-18 23:43:36,854 - langextract.debug - DEBUG - [langextract.inference] RETURN: BaseLanguageModel.__init__ -> None (0.0 ms)
2025-08-18 23:43:36,855 - langextract.debug - DEBUG - [langextract.inference] CALL: BaseLanguageModel.apply_schema(self=<GeminiLanguageModel>, schema_instance=GeminiSchema(...xtractions']}))
2025-08-18 23:43:36,856 - langextract.debug - DEBUG - [langextract.inference] RETURN: BaseLanguageModel.apply_schema -> None (0.0 ms)
DEBUG:absl:Initialized Annotator with prompt:
Extract operating system mentioned by the post author. Make sure to provide all instances of operating systems mentioned in the post.Each post might contain mention of multiple operating systems, extract them all including multple instances of the same operating system.

Examples
Q: I've used Linux for 18

✓ Extraction processing complete



INFO:absl:Finalizing annotation for document ID doc_273c46f8.
INFO:absl:Document annotation completed.


✓ Extracted 3 entities (1 unique types)
  • Time: 1.48s
  • Speed: 393 chars/sec
  • Chunks: 1
Input: Getting the same up and running (and stable!) in windows is a lot of work, especially if you want it to be smooth.. Linux is more configurable, and it is what most of the world's infrastructure and systems run on.And yes, Linux is generally much more efficient than Windows.That said, these days it is mainly preference.For example, I prefer Linux so that I can configure it exactly how I want.I like to use I3wm (A tiling wm) with a custom set of commands combined with oh-my-fish with my custom dotfiles.I have also used MacOS but only minimally, liked it when I used it for a bit.

Extracted entities:
• Distribution: Windows (pos: 49-56)
  Attributes: {'User': 'No'}
• Distribution: Linux (pos: 116-121)
  Attributes: {'User': 'Yes'}
• Distribution: MacOS (pos: 523-528)
  Attributes: {'User': 'Yes'}
AnnotatedDocument(extractions=[Extraction(extraction_class='distribution', extraction_text='W

LangExtract: Saving to linux_extract.jsonl: 1 docs [00:00, 15.08 docs/s]

✓ Saved 1 documents to linux_extract.jsonl



LangExtract: Loading linux_extract.jsonl: 100%|██████████| 1.38k/1.38k [00:00<00:00, 485kB/s]

✓ Loaded 1 documents from linux_extract.jsonl
Interactive visualization saved to linux_extractvisualization.html


In [11]:
#Previous system prompt
system_prompt_old = "Extract operating system being used by the post author"

#Updated system prompt
system_prompt_new = "Extract operating system mentioned by the post author. Make sure to provide all instances of operating systems mentioned in the post." \
"Each post might contain mention of multiple operating systems, extract them all including multple instances of the same operating system."